In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14631, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 14631 (delta 92), reused 97 (delta 54), pack-reused 14478
Receiving objects: 100% (14631/14631), 13.71 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (10072/10072), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 15.8 MB/s 
Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

     |████████████████████████████████| 42 kB 864 kB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 


In [ ]:
# REPLACE with your custom code snippet generated above
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="zkyt15pyUkXSH9tlso0C")
project = rf.workspace("noor-djz41").project("multiuav")
dataset = project.version(2).download("yolov7")

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/yolov3.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/YOLOv5-GVIT.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# darknet53 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   [-1, 1, GhostConv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, GhostBottleneck, [64]],
   [-1, 1, GhostConv, [128, 3, 2]],  # 3-P2/4
   [-1, 2, GhostBottleneck, [128]],
   [-1, 1, GhostConv, [256, 3, 2]],  # 5-P3/8
   [-1, 8, GhostBottleneck, [256]], #6
   [-1, 1, GhostConv, [512, 3, 2]],  # 7-P4/16
   [-1, 8, GhostBottleneck, [512]], #8
   [-1, 1, GhostConv, [1024, 3, 2]],  # 9-P5/32
   [-1, 4, GhostBottleneck, [1024]],  # 10
  ]

# YOLOv3 head
head:
  [[-1, 1, GhostBottleneck, [1024, False]],
   [-1, 1, GhostConv, [512, 1, 1]],
   [-1, 1, GhostConv, [1024, 3, 1]],
   [-1, 1, GhostConv, [512, 1, 1]],
   [-1, 1, GhostConv, [1024, 3, 1]],  # 15 (P5/32-large)

   [-2, 1, GhostConv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 8], 1, Concat, [1]],  # cat backbone P4
   [-1, 1, GhostBottleneck, [512, False]],
   [-1, 1, GhostBottleneck, [512, False]],
   [-1, 1, GhostConv, [256, 1, 1]],
   [-1, 1, GhostConv, [512, 3, 1]],  # 22 (P4/16-medium)

   [-2, 1, GhostConv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P3
   [-1, 1, GhostConv, [256, False]],
   [-1, 2, GhostConv, [256, False]],  # 27 (P3/8-small)

   [[27, 22, 15], 1, Detect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 256 --batch 8 --epochs 500 --optimizer AdamW --hyp data/hyps/hyp.scratch-high.yaml --data {dataset.location}/data.yaml --cfg ./models/YOLOv5-GVIT.yaml --weights '' --name YOLOv5-GVIT_results  --cache

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/YOLOv5-GVIT/results.png', width=1000)  # view results.png

In [ ]:
#########Validate whole dataset
# Validate model and print evaluation metrics
!python val.py --weights runs/train/YOLOv5-GVIT_results/weights/best.pt --data {dataset.location}/data.yaml --img 256 --half

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/
%ls runs/train/YOLOv5-GVIT_results/weights

In [ ]:
######## Test Valid image folder
%cd /content/yolov5/
!python detect.py --weights runs/train/YOLOv5-GVIT_results/weights/best.pt --img 256 --conf 0.25 --source /content/yolov5/Mul-1/valid/images

In [ ]:
##I have created a zip file:
!zip -r /content/YOLOv5-GVIT.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/YOLOv5-GVIT.zip")